In [13]:
import grpc
import adapter_interface_pb2 as pb2
import adapter_interface_pb2_grpc as pb2_grpc

# Konfiguracja kanału gRPC
channel = grpc.insecure_channel('localhost:50051')
stub = pb2_grpc.CloudAdapterStub(channel)

print("✅ Połączono z kanałem gRPC na porcie 50051")

# Nazwy testowe - możesz je zmieniać
TEST_GROUP_NAME = "LabTestGroup_Jupyter"
TEST_LEADERS = ["LeaderJan", "LeaderAnna"]
TEST_USERS = ["StudentKasia", "StudentTomek"]

✅ Połączono z kanałem gRPC na porcie 50051


In [14]:
try:
    # 1. Health Check
    status = stub.GetStatus(pb2.StatusRequest())
    print(f"Server Health: {'🟢 OK' if status.isHealthy else '🔴 BAD'}")

    # 2. Pobranie dostępnych usług (na podstawie plików w config/policies)
    services_resp = stub.GetAvailableServices(pb2.GetAvailableServicesRequest())
    print(f"Dostępne usługi (Resource Types): {services_resp.services}")

except grpc.RpcError as e:
    print(f"❌ Błąd połączenia: {e.code()} - {e.details()}")

Server Health: 🟢 OK
Dostępne usługi (Resource Types): ['dynamoDB', 'ec2', 's3']


In [15]:
try:
    # Na początek dajemy dostęp tylko do S3 i EC2
    initial_resources = ["s3", "ec2"]

    print(f"🏗️ Tworzenie grupy '{TEST_GROUP_NAME}' z zasobami: {initial_resources}...")

    req = pb2.CreateGroupWithLeadersRequest(
        groupName=TEST_GROUP_NAME,
        leaders=TEST_LEADERS,
        resourceTypes=initial_resources
    )

    resp = stub.CreateGroupWithLeaders(req)
    print(f"✅ Sukces! Utworzono grupę: {resp.groupName}")

except grpc.RpcError as e:
    print(f"❌ Błąd tworzenia grupy: {e.details()}")

🏗️ Tworzenie grupy 'LabTestGroup_Jupyter' z zasobami: ['s3', 'ec2']...
✅ Sukces! Utworzono grupę: LabTestGroup_Jupyter


In [16]:
try:
    print(f"👥 Dodawanie studentów {TEST_USERS} do grupy '{TEST_GROUP_NAME}'...")

    req = pb2.CreateUsersForGroupRequest(
        groupName=TEST_GROUP_NAME,
        users=TEST_USERS
    )

    resp = stub.CreateUsersForGroup(req)
    print(f"✅ Odpowiedź serwera: {resp.message}")

except grpc.RpcError as e:
    print(f"❌ Błąd dodawania użytkowników: {e.details()}")

👥 Dodawanie studentów ['StudentKasia', 'StudentTomek'] do grupy 'LabTestGroup_Jupyter'...
✅ Odpowiedź serwera: Pomyślnie przetworzono 2 użytkowników dla grupy 'LabTestGroup_Jupyter'.


In [17]:
try:
    req = pb2.GroupExistsRequest(groupName=TEST_GROUP_NAME)
    resp = stub.GroupExists(req)

    if resp.exists:
        print(f"✅ Grupa '{TEST_GROUP_NAME}' istnieje w IAM.")
    else:
        print(f"❌ Grupa '{TEST_GROUP_NAME}' NIE istnieje (coś poszło nie tak).")

except grpc.RpcError as e:
    print(f"❌ Błąd weryfikacji: {e.details()}")

✅ Grupa 'LabTestGroup_Jupyter' istnieje w IAM.


In [18]:
try:
    # Nowa lista zasobów (doszło DynamoDB)
    # UWAGA: Nie musisz podawać 'region', backend doda go sam dla studentów.
    updated_resources = ["s3", "ec2", "dynamodb"]

    print(f"🛡️ Aktualizacja polityk dla grupy '{TEST_GROUP_NAME}'. Nowe zasoby: {updated_resources}...")

    req = pb2.AssignPoliciesRequest(
        groupName=TEST_GROUP_NAME,
        resourceTypes=updated_resources,
        # userName ignorujemy zgodnie z nową logiką
    )

    resp = stub.AssignPolicies(req)

    if resp.success:
        print(f"✅ Sukces: {resp.message}")
    else:
        print(f"⚠️ Ostrzeżenie: {resp.message}")

except grpc.RpcError as e:
    print(f"❌ Błąd aktualizacji polityk: {e.details()}")

🛡️ Aktualizacja polityk dla grupy 'LabTestGroup_Jupyter'. Nowe zasoby: ['s3', 'ec2', 'dynamodb']...
✅ Sukces: Polityki pomyślnie przypisane do Grupa: LabTestGroup_Jupyter.


In [21]:

user_input = input(f"Czy na pewno usunąć grupę {TEST_GROUP_NAME} i wszystkich userów? (tak/nie): ")

if user_input.lower() == 'tak':
    try:
        print(f"🗑️ Usuwanie grupy '{TEST_GROUP_NAME}'...")
        req = pb2.RemoveGroupRequest(groupName=TEST_GROUP_NAME)
        resp = stub.RemoveGroup(req)

        if resp.success:
            print("✅ Pomyślnie usunięto grupę i użytkowników.")
            print(f"Log: {resp.message}")
            print(f"Usunięci użytkownicy: {resp.removedUsers}")
        else:
            print(f"❌ Błąd usuwania: {resp.message}")

    except grpc.RpcError as e:
        print(f"❌ Błąd RPC: {e.details()}")
else:
    print("Anulowano usuwanie.")

🗑️ Usuwanie grupy 'LabTestGroup_Jupyter'...
✅ Pomyślnie usunięto grupę i użytkowników.
Log: Grupa LabTestGroup_Jupyter usunięta.; Grupa Leaders-LabTestGroup_Jupyter usunięta.
Usunięci użytkownicy: ['StudentKasia-LabTestGroup_Jupyter', 'StudentTomek-LabTestGroup_Jupyter', 'LeaderJan-LabTestGroup_Jupyter', 'LeaderAnna-LabTestGroup_Jupyter']
